[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/neuralangelo-colab/blob/main/neuralangelo_colab.ipynb)

In [ ]:
%cd /content

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/colmap.zip
!unzip /content/colmap.zip -d colmap

!cp -r /content/colmap/lib/. /usr/local/lib
!chmod 755 /content/colmap/bin/colmap
!cp -r /content/colmap/bin/. /usr/local/bin

!pip install -q commentjson addict pynvml wandb
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl

%cd /content
!git clone -b dev --recursive https://github.com/camenduru/neuralangelo

In [ ]:
%cd /content/neuralangelo
!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/lego.mp4 -O /content/lego.mp4
EXPERIMENT="lego"
PATH_TO_VIDEO="/content/lego.mp4"
SKIP_FRAME_RATE=2
SCENE_TYPE="object"
!bash /content/neuralangelo/projects/neuralangelo/scripts/preprocess.sh {EXPERIMENT} {PATH_TO_VIDEO} {SKIP_FRAME_RATE} {SCENE_TYPE}

In [ ]:
!wandb login

In [ ]:
%cd /content/neuralangelo
EXPERIMENT="lego"
GROUP="lego_group"
NAME="lego"
CONFIG=f"/content/neuralangelo/projects/neuralangelo/configs/custom/{EXPERIMENT}.yaml"
GPUS=1
CHECKPOINT_PATH="/content/checkpoint"
!torchrun --nproc_per_node={GPUS} train.py \
    --wandb \
    --wandb_name=neuralangelo \
    --logdir=logs/{GROUP}/{NAME} \
    --config={CONFIG} \
    --show_pbar